In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Coorti 2016-2018 estrazione 2020 02 21 per AI.xlsx to Coorti 2016-2018 estrazione 2020 02 21 per AI.xlsx
Saving esami coorti 2016-2018 - 2020 02 21 per AI.xlsx to esami coorti 2016-2018 - 2020 02 21 per AI.xlsx
User uploaded file "Coorti 2016-2018 estrazione 2020 02 21 per AI.xlsx" with length 6481955 bytes
User uploaded file "esami coorti 2016-2018 - 2020 02 21 per AI.xlsx" with length 15391182 bytes


In [5]:
import pandas as pd
import numpy as np

#Read dataset
features =  pd.read_excel("Coorti 2016-2018 estrazione 2020 02 21 per AI.xlsx")
esami = pd.read_excel("esami coorti 2016-2018 - 2020 02 21 per AI.xlsx")

Siamo interessati al totale dei CFU ottenuti e la media dei voti ottenuti, inoltre abbiamo tenuto soltanto quelli del primo semestre

In [6]:
esami = esami.drop('Cod.Orientamento', axis=1)
esami = esami.drop('Cod.Indirizzo', axis=1)
esami = esami.drop('Cod.Materia', axis=1)
esami = esami.drop('Materia', axis=1)
index_name = esami[(esami['Giorno_esame'].dt.month > 2) & (esami['Giorno_esame'].dt.month < 11)].index
esami.drop(index_name, inplace = True)

        ID_Studente   CdS Giorno_esame  Voto_se_numerico  CFUsuperati
0                 1  8514   2018-02-12              27.0         13.0
1                 1  8514   2018-02-21              27.0         12.0
6                 2  8513   2017-02-02              26.0         12.0
7                 2  8513   2017-02-21              30.0         13.0
9                 3  8871   2017-12-15               NaN          3.0
...             ...   ...          ...               ...          ...
321261        67277  8415   2018-02-01              30.0          6.0
321262        67277  8415   2018-02-22              29.0          8.0
321270        67279  0901   2017-12-19              26.0          8.0
321277        67280  0956   2017-01-23              24.0         12.0
321278        67280  0956   2017-02-20              30.0         12.0

[118826 rows x 5 columns]


Calcoliamo la somma dei CFU per ogni studente la mergiamo con i dati di ogni studente e i valori NaN dei CFU verranno rimpiazzati con 0

In [ ]:
tot_cfu = esami.groupby('ID_Studente')['CFUsuperati'].sum()
features = features.merge(tot_cfu,how='outer', on='ID_Studente')
features['CFUsuperati'] = features['CFUsuperati'].fillna(0)

Calcoliamo la media di voti per ogni studente e la mergiamo con i dati di ogni studente e i valori NaN dei voti verranno rimpiazzati con 0

In [ ]:
esami = esami.drop('CFUsuperati', axis=1)
esami = esami.dropna()
media_voti = esami.groupby('ID_Studente')['Voto_se_numerico'].mean()
features = features.merge(media_voti,how='outer', on='ID_Studente')
features['Voto_se_numerico'] = features['Voto_se_numerico'].fillna(0)

Rimpiazziamo i codici relativi a determinate feature con i nomi delle feature, così da poter utilizzare la funzione get_dummies() ed ottenere un one-hot-encoding

In [ ]:
features["Diploma_scuola_superiore"] = features["Diploma_scuola_superiore"].map({1: "Classico", 2: "Scientifico", 3: "Linguistico", 4: "Magistrale", 5: "Artistico", 6: "Tecnico", 7: "Professionale", 8: "Altro_italiano", 9: "Estero", 99: "Non_disponibile"})
features["area_geografica_scuolasuperiore"] = features["area_geografica_scuolasuperiore"].map({1: "Emilia_romagna", 2: "Nord", 3: "Centro", 4: "Sud_isole", 5: "Estero", 99: "Non_disponibile"})
features["area_geografica_residenza"] = features["area_geografica_residenza"].map({1: "Emilia_romagna", 2: "Nord", 3: "Centro", 4: "Sud_isole", 5: "Estero", 99: "Non_disponibile"})
features["Ambito"] = features["Ambito"].map({1: "Economia", 2: "Farmacia", 3: "Giurisprudenza", 4: "Ingegneria", 5: "Lingue", 6: "Medicina", 7: "Veterinaria", 8: "Psicologia", 9: "Scienze", 10: "Scienze_agroalimanetari", 11: "Scienze_educazione_formazione", 12: "Scienze_motorie", 13: "Scienze_politiche", 14: "Scienze_statistiche", 15: "Sociologia", 16: "Studi_umanistici"})
features = pd.get_dummies(features)

Abbiamo sostituito i valori NaN della feature "voto_scuola_superiore" con la media.

In [ ]:
mean_value = features['voto_scuola_superiore'].mean()
features['voto_scuola_superiore'] = features['voto_scuola_superiore'].fillna(mean_value)

Inoltre sono state droppate le colonne non utili ai fini dell'addestramento, ed infine eliminate le tuple contente almeno un NaN

In [ ]:
features= features.drop('ID_Studente', axis = 1)
features= features.drop('Classe_ISEE', axis = 1)
features= features.drop('DataNascita', axis = 1)
features = features.dropna()

Abbiamo bilanciato il dataset con approccio downsampling pareggiando il numero di dropout e non droput

In [ ]:
#Data preprocessing
dropout = pd.DataFrame()
noDropout = pd.DataFrame()

for ind in features.index:
    studente = pd.DataFrame(features.loc[[ind]])
    if features["Abbandoni"][ind] == 1:
        dropout = pd.concat([studente, dropout], ignore_index=True, axis = 0)
    else:
        noDropout = pd.concat([studente, noDropout], ignore_index=True, axis = 0)

#Bilanciamento dataset
final = pd.DataFrame()
for ind in dropout.index:
    studente_drop = pd.DataFrame(dropout.loc[[ind]])
    studente_noDrop = pd.DataFrame(noDropout.loc[[ind]])
    final = pd.concat([studente_drop, final], ignore_index=True, axis = 0)
    final = pd.concat([studente_noDrop, final], ignore_index=True, axis = 0)

print(len(final))

In [ ]:
from sklearn.model_selection import train_test_split as tts
#Dividing the data into attributes and labels
X = final.drop('Abbandoni', axis=1) #sostituire final con features
y = final['Abbandoni'] #sostituire final con features
#Dividing data into training and test sets
X_train, X_test, y_train, y_test = tts(X, y, test_size = 0.20, random_state=109)

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error,r2_score
from sklearn import metrics

X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.2)
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape